### 트랜스포머 애플리케이션 둘러보기

In [17]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [18]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

* 텍스트 분류

In [19]:
from transformers import pipeline

In [20]:
classifier = pipeline('text-classification') # 관심작업 이름 전달
# text-classification : 주로 감성분석, 다중 분류, 다중 레이블 분류

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [21]:
import pandas as pd

outputs = classifier(text)
outputs # 각 예측은 하나의 dict형

[{'label': 'NEGATIVE', 'score': 0.9015461802482605}]

In [22]:
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


* 개체명 인식

In [23]:
ner_tagger = pipeline('ner', aggregation_strategy = 'simple') # 모델 예측에 따라 단어 그룹화

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
C:\Users\knuyh\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:982: UserWarning: Not enough free disk space to download the file. The expected file size is: 1334.40 MB. The target location C:\Users\knuyh\.cache\huggingface\hub\models--dbmdz--bert-large-cased-finetuned-conll03-english\blobs only has 20.64 MB free disk space.
  warnings.warn(


model.safetensors:  16%|#5        | 210M/1.33G [00:00<?, ?B/s]

C:\Users\knuyh\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:982: UserWarning: Not enough free disk space to download the file. The expected file size is: 1334.40 MB. The target location C:\Users\knuyh\.cache\huggingface\hub\models--dbmdz--bert-large-cased-finetuned-conll03-english\blobs only has 10.15 MB free disk space.
  warnings.warn(


model.safetensors:  17%|#6        | 220M/1.33G [00:00<?, ?B/s]

model.safetensors:  17%|#6        | 220M/1.33G [00:00<?, ?B/s]

model.safetensors:  17%|#6        | 220M/1.33G [00:00<?, ?B/s]

ValueError: Could not load model dbmdz/bert-large-cased-finetuned-conll03-english with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForTokenClassification'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForTokenClassification'>, <class 'transformers.models.bert.modeling_bert.BertForTokenClassification'>, <class 'transformers.models.bert.modeling_tf_bert.TFBertForTokenClassification'>). See the original errors:

while loading with AutoModelForTokenClassification, an error is thrown:
Traceback (most recent call last):
  File "C:\Users\knuyh\anaconda3\Lib\site-packages\transformers\pipelines\base.py", line 283, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\knuyh\anaconda3\Lib\site-packages\transformers\models\auto\auto_factory.py", line 564, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^

In [ ]:
outputs = ner_tagger(text)
pd.DataFrame(outputs)
# ORG : 조직, LOC : 위치, PER : 사람, MISC : 그 외

* 질문 답변

In [ ]:
reader = pipeline('question-answering')
question = 'What does the customer want?'
outputs = reader(question = question, context = text)
pd.DataFrame(outputs)

* 요약

In [ ]:
summarizer = pipeline('summarization')
outputs = summarizer(text, max_length = 60, clean_up_tokenization_spaces = True)
print(outputs[0]['summary_text'])

* 번역

In [ ]:
translator = pipeline('translation_en_to_de',
                     model = 'Helsinki-NLP/opus-mt-en-de') # 영어 -> 독어
outputs = translator(text, clean_up_tokenization_spaces = True, min_length = 100)
print(outputs[0]['translation_text'])

* 텍스트 생성

In [ ]:
generator = pipeline('text-generation')
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
# 자동완성기능
prompt = text + "\n\nCustomer service response:\n" + response

In [ ]:
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])